In [1]:
import datajoint as dj
import numpy as np
import time
from tqdm import tqdm

import matplotlib.pyplot as plt
import ipyvolume.pylab as p3

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [2]:
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')

Connecting cpapadop@10.28.0.34:3306


In [3]:
fetched_mesh = ta3p100.Decimation35.fetch(limit=1, as_dict=True)[0]

# Skeleton

In [ ]:
# Once I make several, small voxels it would be quite easy to turn that into a skeleton structure by just connecting voxels
# that are next to each other.

# I could use the centroids of the voxels as bone points, and also downsample the bone structure by checking to see how far
# a bone vector strays from the centroids of adjacent voxels, or rather the downstream voxels from a starting voxel.

# If I can scan across the whole mesh and start building regression lines... and correcting and branching off... hmm that might be possible.
# Basically I would start a bone at a vertex, and continue that bone as I scan through the mesh across one axis. If I find a vertex far enough
# away from my current bone then I either start a new bone OR I branch off from the current bone.
# I think that I will basically grant membership of vertices to each bone in order to create the skeleton structure. So I need to keep track
# of which vertices are assigned to each bone. Probably using a dict, and from there I can just append vertices to the bone index. This will
# also allow me to compare 2 bones (and their vertices) with each other by looking at 2 bone indices in the dict.
# I should also make a verification function to see how close the vertices are to their assigned bones.
# Depending on how I merge bones, I might get spines also defined as bones, which might be beneficial.
# Essentially I'm going through a decision tree:
# 1. If I find a vertex on its own, I create a bone.
# 2. If I find a vertex near another vertex, I keep the bone growing.
# 

class Skeleton:
    _bones = list()
    
    def __init__(self, vertices, triangles):
        self._original_vertices = vertices
        self._original_triangles = triangles
        
        self._vertices = vertices
        self._triangles = triangles
    
    @property
    def vertices(self):
        return self._vertices
    
    @property
    def triangles(self):
        return self._triangles
    
    @property
    def _nonisolated_vertices(self):
        unique_vertex_idx = np.unique(self.triangles)
        return self.vertices[unique_vertex_idx]
    
    @property
    def _sorted_vertices(self):
        """
        Returns sorted nonisolated vertices.
        """
        verts = self._nonisolated_vertices
        sorted_idx = verts.T[2].argsort()
        return verts[sorted_idx]
    
    @property
    def bbox(self):
        return np.array([(np.min(axis), np.max(axis)) for axis in self.vertices.T]) # Should I use nonisolated vertices though?
    
    def plot_mesh(self, width=1024, height=1024, **kwargs):
        p3.figure(width=width, height=height)
        p3.plot_trisurf(*self.vertices.T/1000, self.triangles, **kwargs)
        p3.squarelim()
        p3.show()
    
    def plot_verts(self, width=1024, height=1024, targeted_verts=None, **kwargs):
        if targeted_verts is None:
            verts = self.vertices
        else:
            verts = targeted_verts
        
        p3.figure(width=width, height=height)
        p3.scatter(*verts.T/1000, **kwargs)
        p3.squarelim()
        p3.show()
        
    def thick_plane(self, num_planes=50):
        z_space = np.linspace(*self.bbox[2], num=num_planes)

        starting_idx = 0
        partition_edge_idx = list([starting_idx])
        verts = self._sorted_vertices
        for j, z_edge in enumerate(z_space[1:-1]):
            for i, vert in enumerate(verts.T[2][starting_idx:]):
                if vert > z_edge:
                    starting_idx += i
                    partition_edge_idx.append(starting_idx)
                    break
        partition_edge_idx.append(-1) #(len(verts) - 1)
        
        return np.array(partition_edge_idx)
    
    @property
    def bones(self):
        return self._bones
    
    def add_bone(self, starting_point, ending_point):
        # The bones should probably be connected in some way? Though a branching bone might not want to be connected.
        self._bones.append((starting_point, ending_point))
        
    def scan(self, axis='x'):        
        if axis.lower() == 'x':
            axis_idx = 0
        elif axis.lower() == 'y':
            axis_idx = 1
        elif axis.lower() == 'z':
            axis_idx = 2
        else:
            raise ValueError("Invalid value for axis, choose between 'x', 'y', and 'z'.")
            
        verts = self._sorted_vertices
        
        bone_membership = dict()
        for vert in verts:
            

In [ ]:
skel = Skeleton()

# Voxel

In [222]:
class Mesh:
    def __init__(self, vertices, triangles, ignore_isolated_vertices=True):
        self._original_vertices = vertices
        self._original_triangles = triangles
        
        self._vertices = vertices
        self._triangles = triangles
        self._ignore_isolated = ignore_isolated_vertices
        
        self._voxels = list()
    
    class Voxel: # Override some operators to allow direct manipulation to the bbox inside.
        """
        If I really want Voxels that are cubes, I can push the boundaries of the Mesh bounding box to fit what I need (to allow the cubes to fit edge to edge).
        """
        def __init__(self, bbox):
            self._bbox = bbox
            
        @property
        def bbox(self):
            return self._bbox
        
        @bbox.setter
        def bbox(self, bbox):
            if not isinstance(bbox, np.ndarray):
                bbox = np.array(bbox)
            
            if bbox.shape == (3, 2):
                self._bbox = bbox
            else:
                raise ValueError("Bounding box is not in required form which is: array-like with shape (3, 2).")
            
        @property
        def centroid(self):
            return self.bbox.T.mean(axis=0)
        
        def __str__(self):
            return str(self.bbox)
    
    @property
    def voxels(self):
        return self._voxels
    
    @voxels.setter
    def voxels(self, voxels):
        self._voxels = voxels
    
    @property
    def vertices(self): # Could "potentially" have the ignore_isolated_vertices_check in here... but it would mess up indices, so put in
        # another property entirely.
        return self._vertices
    
    @property
    def triangles(self):
        return self._triangles
    
    @property
    def _nonisolated_vertices(self):
        unique_vertex_idx = np.unique(self.triangles)
        return self.vertices[unique_vertex_idx]
    
    @property
    def bbox(self):
        return np.array([(np.min(axis), np.max(axis)) for axis in self.vertices.T])
    
    @staticmethod
    def get_bbox(vertices):
        return np.array([(np.min(axis), np.max(axis)) for axis in vertices.T])
    
        
    def plot_mesh(self, width=1024, height=1024, **kwargs):
        p3.figure(width=width, height=height)
        p3.plot_trisurf(*self.vertices.T/1000, self.triangles, **kwargs)
        p3.squarelim()
        p3.show()
    
    def generate_voxel_lines(self):
        bboxes = np.array([voxel.bbox for voxel in self.voxels])
        return np.array([[[xs[0], ys[0], zs[0]],
                         [xs[0], ys[0], zs[1]],
                         [xs[0], ys[1], zs[1]],
                         [xs[0], ys[1], zs[0]],
                         [xs[0], ys[0], zs[0]],

                         [xs[1], ys[0], zs[0]],
                         [xs[1], ys[0], zs[1]],
                         [xs[1], ys[1], zs[1]],
                         [xs[1], ys[1], zs[0]],
                         [xs[1], ys[0], zs[0]],

                         [xs[0], ys[0], zs[0]],
                         [xs[0], ys[0], zs[1]],
                         [xs[1], ys[0], zs[1]],
                         [xs[1], ys[0], zs[0]],
                         [xs[0], ys[0], zs[0]],

                         [xs[0], ys[1], zs[0]],
                         [xs[0], ys[1], zs[1]],
                         [xs[1], ys[1], zs[1]],
                         [xs[1], ys[1], zs[0]],
                         [xs[0], ys[1], zs[0]]] for xs, ys, zs in bboxes])
    
    def plot_voxels(self, width=1024, height=1024):
        fig = p3.figure(width=width, heigh=height)
        [p3.plot(*lines.T/1000, color='blue') for lines in self.generate_voxel_lines()]
#         p3.scatter(*np.array([voxel.centroid for voxel in self.voxels]).T/1000, marker='sphere')
        p3.squarelim()
        p3.show()
        
#         return fig
    
    def plot_mesh_and_voxels(self, width=1024, height=1024, **kwargs):
        p3.figure(width=width, height=height)
        p3.plot_trisurf(*self.vertices.T/1000, self.triangles, **kwargs)
        [p3.plot(*lines.T/1000, color='blue') for lines in self.generate_voxel_lines()]
#         p3.scatter(*np.arrayy([voxel.centroid for voxel in self.voxels]).T/1000, marker='sphere', size=0.5, color='blue')
        p3.squarelim()
        p3.show()    
    
    def merge_touching_voxels(self):
        raise NotImplementedError
        
    def debug(self, voxel_side_length=1000):
        n_splits = np.array([((axis[1] - axis[0]) / voxel_side_length) + 1 for axis in self.bbox], np.uint32)

        x_split, y_split, z_split = [np.linspace(minimum, maximum, num=axis_splits)
                                     for (minimum, maximum), axis_splits in zip(self.bbox, n_splits)]
        
        bboxes = list()
        x_pairs = np.array([(x_split[i], x_split[i+1]) for i in range(len(x_split) - 1)])
        y_pairs = np.array([(y_split[i], y_split[i+1]) for i in range(len(y_split) - 1)])
        z_pairs = np.array([(z_split[i], z_split[i+1]) for i in range(len(z_split) - 1)])
        for xs in x_pairs:
            for ys in y_pairs:
                for zs in z_pairs:
                    bboxes.append((xs, ys, zs))
        bboxes = np.array(bboxes)
        
        bbox_lines = np.array([[[xs[0], ys[0], zs[0]],
                         [xs[0], ys[0], zs[1]],
                         [xs[0], ys[1], zs[1]],
                         [xs[0], ys[1], zs[0]],
                         [xs[0], ys[0], zs[0]],

                         [xs[1], ys[0], zs[0]],
                         [xs[1], ys[0], zs[1]],
                         [xs[1], ys[1], zs[1]],
                         [xs[1], ys[1], zs[0]],
                         [xs[1], ys[0], zs[0]],

                         [xs[0], ys[0], zs[0]],
                         [xs[0], ys[0], zs[1]],
                         [xs[1], ys[0], zs[1]],
                         [xs[1], ys[0], zs[0]],
                         [xs[0], ys[0], zs[0]],

                         [xs[0], ys[1], zs[0]],
                         [xs[0], ys[1], zs[1]],
                         [xs[1], ys[1], zs[1]],
                         [xs[1], ys[1], zs[0]],
                         [xs[0], ys[1], zs[0]]] for xs, ys, zs in bboxes])
        
        p3.figure(width=1024, height=1024)
        p3.plot_trisurf(*self.vertices.T/1000, self.triangles)
        [p3.plot(*lines.T/1000, color='blue') for lines in bbox_lines]
        p3.squarelim()
        p3.show()
        
        return bboxes
    
    def restrict_bboxes(self, voxel_side_length=1000): # Can improve the cubimetrics (fake word for splitting it into cubes)
        nonisolated_vertices = self._nonisolated_vertices
        
        # I should try rounding the mesh bbox up to the nearest level that is evenly divisible by the voxel_side_length
        # that way I can actually make it split into cubes nicely.
        
        n_splits = np.array([((axis[1] - axis[0]) / voxel_side_length) + 1 for axis in self.bbox], np.uint32)
#         n_splits = [11] * 3
    
        x_edges, y_edges, z_edges = [np.linspace(minimum, maximum, num=axis_splits)[1:-1]
                                     for (minimum, maximum), axis_splits in zip(self.bbox, n_splits)]
                
        bboxes = list()
        
        sorted_idx = nonisolated_vertices.T[0].argsort()
        x_verts = nonisolated_vertices[sorted_idx]
        splitter = x_verts.T[0].searchsorted(x_edges)
        x_split = np.split(x_verts, splitter)
        for x_block in x_split:
            if len(x_block) > 0:
                sorted_idx = x_block.T[1].argsort()
                y_verts = x_block[sorted_idx]
                splitter = y_verts.T[1].searchsorted(y_edges)
                y_split = np.split(y_verts, splitter)
                if len(y_split) > 0:
                    for y_block in y_split:
                        if len(y_block) > 0:
                            sorted_idx = y_block.T[2].argsort()
                            z_verts = y_block[sorted_idx]
                            splitter = z_verts.T[2].searchsorted(z_edges)
                            z_split = np.split(z_verts, splitter)
                            if len(z_split) > 0:
                                for z_block in z_split:
                                    if len(z_block) > 0:
                                        bboxes.append(self.get_bbox(z_block))
            
        return bboxes
    
    def initialize_voxels(self, voxel_side_length=1000):
        bboxes = self.restrict_bboxes(voxel_side_length=voxel_side_length)
        self._voxels = [self.Voxel(bbox) for bbox in bboxes]
        return self.voxels

In [234]:
mesh = Mesh(fetched_mesh['vertices'], fetched_mesh['triangles'])
start = time.time()
mesh.initialize_voxels(5000)
time.time() - start, len(mesh.voxels)

(0.22087335586547852, 751)

In [230]:
mesh.plot_mesh_and_voxels()

In [5]:
unique_vertex_idx = np.unique(fetched_mesh['triangles'])
nonisolated_vertices = fetched_mesh['vertices'][unique_vertex_idx]
bbox = Mesh.get_bbox(nonisolated_vertices)
voxel_side_length = 10000

# This is the actual good solution goddamnit

In [198]:
edge_list = np.arange(0, 16, 5)
vert_list = np.arange(16)
edge_list, vert_list

(array([ 0,  5, 10, 15]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]))

In [202]:
vert_list = np.array([ 0, 1, 6, 7, 8, 9, 10, 13, 14])

In [203]:
A = vert_list.searchsorted(edge_list[1:-1])
B = np.split(vert_list, A)

In [204]:
for b in B:
    print(b)

[0 1]
[6 7 8 9]
[10 13 14]


In [37]:
start_idx = 0
for edge in edge_list:
    for i, vert in enumerate(vert_list[start_idx:]):
        if vert > edge:
            print(vert, edge)
            break

1 0
6 5
11 10


In [55]:
a_start_idx = 0
for a_edge in edge_list:
    for i, a_vert in enumerate(vert_list[a_start_idx:]):
        if a_vert > a_edge:
            a_start_idx += i
            
            b_start_idx = 0
            for b_edge in edge_list:
                for j, b_vert in enumerate(vert_list[b_start_idx:]):
                    if b_vert > b_edge:
                        b_start_idx += j
                        
                        c_start_idx = 0
                        for c_edge in edge_list:
                            for k, c_vert in enumerate(vert_list[c_start_idx:]):
                                if c_vert > c_edge:
                                    c_start_idx += k
                                    print()
                                    print(a_vert, b_vert, c_vert)
                                    print(a_edge, b_edge, c_edge)
                                    print()
                                    break
                        break
            break

A 0 1
BB 0 1
CCC 0 1

1 1 1
0 0 0

CCC 1 6

1 1 6
0 0 5

CCC 6 11

1 1 11
0 0 10

BB 1 6
CCC 0 1

1 6 1
0 5 0

CCC 1 6

1 6 6
0 5 5

CCC 6 11

1 6 11
0 5 10

BB 6 11
CCC 0 1

1 11 1
0 10 0

CCC 1 6

1 11 6
0 10 5

CCC 6 11

1 11 11
0 10 10

A 1 6
BB 0 1
CCC 0 1

6 1 1
5 0 0

CCC 1 6

6 1 6
5 0 5

CCC 6 11

6 1 11
5 0 10

BB 1 6
CCC 0 1

6 6 1
5 5 0

CCC 1 6

6 6 6
5 5 5

CCC 6 11

6 6 11
5 5 10

BB 6 11
CCC 0 1

6 11 1
5 10 0

CCC 1 6

6 11 6
5 10 5

CCC 6 11

6 11 11
5 10 10

A 6 11
BB 0 1
CCC 0 1

11 1 1
10 0 0

CCC 1 6

11 1 6
10 0 5

CCC 6 11

11 1 11
10 0 10

BB 1 6
CCC 0 1

11 6 1
10 5 0

CCC 1 6

11 6 6
10 5 5

CCC 6 11

11 6 11
10 5 10

BB 6 11
CCC 0 1

11 11 1
10 10 0

CCC 1 6

11 11 6
10 10 5

CCC 6 11

11 11 11
10 10 10



In [59]:
for pair in edge_pairs:
    for i, vert in enumerate(vert_list):
        if (vert >= pair[0]) and (vert <= pair[1]):
            print(vert, pair)

0 [0 5]
1 [0 5]
2 [0 5]
3 [0 5]
4 [0 5]
5 [0 5]
5 [ 5 10]
6 [ 5 10]
7 [ 5 10]
8 [ 5 10]
9 [ 5 10]
10 [ 5 10]
10 [10 15]
11 [10 15]
12 [10 15]
13 [10 15]
14 [10 15]
15 [10 15]


In [65]:
for a_pair in edge_pairs:
    for i, a_vert in enumerate(vert_list):
        if (a_vert >= a_pair[0]) and (a_vert <= a_pair[1]):
            
            for b_pair in edge_pairs:
                for i, b_vert in enumerate(vert_list):
                    if (b_vert >= b_pair[0]) and (b_vert <= b_pair[1]):
                        
                        for c_pair in edge_pairs:
                            for i, c_vert in enumerate(vert_list):
                                if (c_vert >= c_pair[0]) and (c_vert <= c_pair[1]):
                                    print(a_vert, b_vert, c_vert)
#                                 else:
#                                     break
#                     else:
#                         break
#         else:
#             break

0 0 0
0 0 1
0 0 2
0 0 3
0 0 4
0 0 5
0 0 5
0 0 6
0 0 7
0 0 8
0 0 9
0 0 10
0 0 10
0 0 11
0 0 12
0 0 13
0 0 14
0 0 15
0 1 0
0 1 1
0 1 2
0 1 3
0 1 4
0 1 5
0 1 5
0 1 6
0 1 7
0 1 8
0 1 9
0 1 10
0 1 10
0 1 11
0 1 12
0 1 13
0 1 14
0 1 15
0 2 0
0 2 1
0 2 2
0 2 3
0 2 4
0 2 5
0 2 5
0 2 6
0 2 7
0 2 8
0 2 9
0 2 10
0 2 10
0 2 11
0 2 12
0 2 13
0 2 14
0 2 15
0 3 0
0 3 1
0 3 2
0 3 3
0 3 4
0 3 5
0 3 5
0 3 6
0 3 7
0 3 8
0 3 9
0 3 10
0 3 10
0 3 11
0 3 12
0 3 13
0 3 14
0 3 15
0 4 0
0 4 1
0 4 2
0 4 3
0 4 4
0 4 5
0 4 5
0 4 6
0 4 7
0 4 8
0 4 9
0 4 10
0 4 10
0 4 11
0 4 12
0 4 13
0 4 14
0 4 15
0 5 0
0 5 1
0 5 2
0 5 3
0 5 4
0 5 5
0 5 5
0 5 6
0 5 7
0 5 8
0 5 9
0 5 10
0 5 10
0 5 11
0 5 12
0 5 13
0 5 14
0 5 15
0 5 0
0 5 1
0 5 2
0 5 3
0 5 4
0 5 5
0 5 5
0 5 6
0 5 7
0 5 8
0 5 9
0 5 10
0 5 10
0 5 11
0 5 12
0 5 13
0 5 14
0 5 15
0 6 0
0 6 1
0 6 2
0 6 3
0 6 4
0 6 5
0 6 5
0 6 6
0 6 7
0 6 8
0 6 9
0 6 10
0 6 10
0 6 11
0 6 12
0 6 13
0 6 14
0 6 15
0 7 0
0 7 1
0 7 2
0 7 3
0 7 4
0 7 5
0 7 5
0 7 6
0 7 7
0 7 8
0 7 9
0 7 10
0 7 10


5 2 2
5 2 3
5 2 4
5 2 5
5 2 5
5 2 6
5 2 7
5 2 8
5 2 9
5 2 10
5 2 10
5 2 11
5 2 12
5 2 13
5 2 14
5 2 15
5 3 0
5 3 1
5 3 2
5 3 3
5 3 4
5 3 5
5 3 5
5 3 6
5 3 7
5 3 8
5 3 9
5 3 10
5 3 10
5 3 11
5 3 12
5 3 13
5 3 14
5 3 15
5 4 0
5 4 1
5 4 2
5 4 3
5 4 4
5 4 5
5 4 5
5 4 6
5 4 7
5 4 8
5 4 9
5 4 10
5 4 10
5 4 11
5 4 12
5 4 13
5 4 14
5 4 15
5 5 0
5 5 1
5 5 2
5 5 3
5 5 4
5 5 5
5 5 5
5 5 6
5 5 7
5 5 8
5 5 9
5 5 10
5 5 10
5 5 11
5 5 12
5 5 13
5 5 14
5 5 15
5 5 0
5 5 1
5 5 2
5 5 3
5 5 4
5 5 5
5 5 5
5 5 6
5 5 7
5 5 8
5 5 9
5 5 10
5 5 10
5 5 11
5 5 12
5 5 13
5 5 14
5 5 15
5 6 0
5 6 1
5 6 2
5 6 3
5 6 4
5 6 5
5 6 5
5 6 6
5 6 7
5 6 8
5 6 9
5 6 10
5 6 10
5 6 11
5 6 12
5 6 13
5 6 14
5 6 15
5 7 0
5 7 1
5 7 2
5 7 3
5 7 4
5 7 5
5 7 5
5 7 6
5 7 7
5 7 8
5 7 9
5 7 10
5 7 10
5 7 11
5 7 12
5 7 13
5 7 14
5 7 15
5 8 0
5 8 1
5 8 2
5 8 3
5 8 4
5 8 5
5 8 5
5 8 6
5 8 7
5 8 8
5 8 9
5 8 10
5 8 10
5 8 11
5 8 12
5 8 13
5 8 14
5 8 15
5 9 0
5 9 1
5 9 2
5 9 3
5 9 4
5 9 5
5 9 5
5 9 6
5 9 7
5 9 8
5 9 9
5 9 10
5 9 10
5 9 11
5 9 1

9 11 15
9 12 0
9 12 1
9 12 2
9 12 3
9 12 4
9 12 5
9 12 5
9 12 6
9 12 7
9 12 8
9 12 9
9 12 10
9 12 10
9 12 11
9 12 12
9 12 13
9 12 14
9 12 15
9 13 0
9 13 1
9 13 2
9 13 3
9 13 4
9 13 5
9 13 5
9 13 6
9 13 7
9 13 8
9 13 9
9 13 10
9 13 10
9 13 11
9 13 12
9 13 13
9 13 14
9 13 15
9 14 0
9 14 1
9 14 2
9 14 3
9 14 4
9 14 5
9 14 5
9 14 6
9 14 7
9 14 8
9 14 9
9 14 10
9 14 10
9 14 11
9 14 12
9 14 13
9 14 14
9 14 15
9 15 0
9 15 1
9 15 2
9 15 3
9 15 4
9 15 5
9 15 5
9 15 6
9 15 7
9 15 8
9 15 9
9 15 10
9 15 10
9 15 11
9 15 12
9 15 13
9 15 14
9 15 15
10 0 0
10 0 1
10 0 2
10 0 3
10 0 4
10 0 5
10 0 5
10 0 6
10 0 7
10 0 8
10 0 9
10 0 10
10 0 10
10 0 11
10 0 12
10 0 13
10 0 14
10 0 15
10 1 0
10 1 1
10 1 2
10 1 3
10 1 4
10 1 5
10 1 5
10 1 6
10 1 7
10 1 8
10 1 9
10 1 10
10 1 10
10 1 11
10 1 12
10 1 13
10 1 14
10 1 15
10 2 0
10 2 1
10 2 2
10 2 3
10 2 4
10 2 5
10 2 5
10 2 6
10 2 7
10 2 8
10 2 9
10 2 10
10 2 10
10 2 11
10 2 12
10 2 13
10 2 14
10 2 15
10 3 0
10 3 1
10 3 2
10 3 3
10 3 4
10 3 5
10 3 5
10 3 6
10 3 

14 6 12
14 6 13
14 6 14
14 6 15
14 7 0
14 7 1
14 7 2
14 7 3
14 7 4
14 7 5
14 7 5
14 7 6
14 7 7
14 7 8
14 7 9
14 7 10
14 7 10
14 7 11
14 7 12
14 7 13
14 7 14
14 7 15
14 8 0
14 8 1
14 8 2
14 8 3
14 8 4
14 8 5
14 8 5
14 8 6
14 8 7
14 8 8
14 8 9
14 8 10
14 8 10
14 8 11
14 8 12
14 8 13
14 8 14
14 8 15
14 9 0
14 9 1
14 9 2
14 9 3
14 9 4
14 9 5
14 9 5
14 9 6
14 9 7
14 9 8
14 9 9
14 9 10
14 9 10
14 9 11
14 9 12
14 9 13
14 9 14
14 9 15
14 10 0
14 10 1
14 10 2
14 10 3
14 10 4
14 10 5
14 10 5
14 10 6
14 10 7
14 10 8
14 10 9
14 10 10
14 10 10
14 10 11
14 10 12
14 10 13
14 10 14
14 10 15
14 10 0
14 10 1
14 10 2
14 10 3
14 10 4
14 10 5
14 10 5
14 10 6
14 10 7
14 10 8
14 10 9
14 10 10
14 10 10
14 10 11
14 10 12
14 10 13
14 10 14
14 10 15
14 11 0
14 11 1
14 11 2
14 11 3
14 11 4
14 11 5
14 11 5
14 11 6
14 11 7
14 11 8
14 11 9
14 11 10
14 11 10
14 11 11
14 11 12
14 11 13
14 11 14
14 11 15
14 12 0
14 12 1
14 12 2
14 12 3
14 12 4
14 12 5
14 12 5
14 12 6
14 12 7
14 12 8
14 12 9
14 12 10
14 12 10
14 12 11
1

In [134]:
vert_list, edge_list

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]),
 array([ 0,  5, 10, 15]))

In [139]:
vert_list = np.array([ 0, 1, 6, 7, 8, 9, 10, 13, 14])

In [ ]:
edge_idx = 1
vert_idx = 0

for i, vert in enumerate(vert_list):
    if vert > edge_list[edge_idx]:
        print(vert_list[vert_idx:i]) 
        edge_idx += 1
        vert_idx = i
        if edge_idx == len(edge_list) - 1:
            print(vert_list[vert_idx:]) 

### This is a good iteration

In [137]:
# Starting point for edges
edge_idx = 1

# Starting point for indices
vert_idx = 0

# Iterate through every vertex. Keep track of which vertex I'm on.
for i, vert in enumerate(vert_list):
    # Next stop if I reach the first edge.
    if vert > edge_list[edge_idx]:
        print(vert_list[vert_idx:i]) 
        # Now I move to the next edge in the list. But I need to ensure I get the final indices too.
        edge_idx += 1
        # I also increase the starting index
        vert_idx = i
        # I am missing the final section.
        if edge_idx == len(edge_list) - 1:
            print(vert_list[vert_idx:]) 

[0 1]
[ 6  7  8  9 10]
[13 14]


### This aims to be more elegant with the final case

In [146]:
edge_list = np.arange(0, 16, 5)
vert_list = np.arange(16)
edge_list, vert_list

(array([ 0,  5, 10, 15]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]))

In [157]:
# edge_list = np.array([ 0,  5, 10])
vert_list = np.array([ 0, 1, 6, 7, 8, 9, 10, 13, 14])
edge_list, vert_list

(array([ 0,  5, 10, 15]), array([ 0,  1,  6,  7,  8,  9, 10, 13, 14]))

In [180]:
# Starting point for edges
edge_idx = 1

# Starting point for indices
vert_idx = 0

# Iterate through every vertex. Keep track of which vertex I'm on.
for i, vert in enumerate(vert_list):
    # Next, stop if I reach the first edge.
    if vert > edge_list[edge_idx]:
        print(edge_list[edge_idx])
        print(vert_list[vert_idx:i], "\n")
        # Now I move to the next edge in the list. But I need to ensure I get the final indices too.
        edge_idx += 1
        # I also increase the starting index
        vert_idx = i
        # I am missing the final section.
        if edge_idx == len(edge_list) - 1:
            print(edge_list[edge_idx])
            print(vert_list[vert_idx:])

5
[0 1] 

10
[ 6  7  8  9 10] 

15
[13 14]


In [190]:
A = np.array([0.1, 3.5, 6.5, 7.9, 11.4, 12.0, 22.3, 24.5, 26.7, 29.9])
split_at = A.searchsorted([10, 20])
B = np.split(A, split_at)

In [191]:
B

[array([0.1, 3.5, 6.5, 7.9]),
 array([11.4, 12. ]),
 array([22.3, 24.5, 26.7, 29.9])]

In [195]:
A = vert_list.searchsorted(edge_list[1:-1])
B = np.split(vert_list, A)

In [196]:
B

[array([0, 1]), array([6, 7, 8, 9]), array([10, 13, 14])]

In [ ]:
# I will split up the vert list before hand.
for i, vert in enumerate(vert_list):
    

In [30]:
# %%debug

# I should try rounding the mesh bbox up to the nearest level that is evenly divisible by the voxel_side_length
# that way I can actually make it split into cubes nicely.
start = time.time()

n_splits = np.array([((axis[1] - axis[0]) / voxel_side_length) + 1 for axis in bbox], np.uint32)
#         n_splits = [11] * 3

x_edges, y_edges, z_edges = [np.linspace(minimum, maximum, num=axis_splits)[1:-1]
                             for (minimum, maximum), axis_splits in zip(bbox, n_splits)]

print(len(x_edges), len(y_edges), len(z_edges))

bboxes = list()

 # X iterate start
sorted_idx = nonisolated_vertices.T[0].argsort()
x_verts = nonisolated_vertices[sorted_idx]

x_start_idx = 0 #100000 # Still need to work out for things that don't start at the normal place
for x_edge in x_edges:
    for i, x_vert in enumerate(x_verts[x_start_idx:]):
        if x_vert[0] > x_edge:
            
            
            temp = x_start_idx
            x_start_idx += i
            restricted_verts = x_verts[temp:x_start_idx]
#             x_start_idx += i

            # Y iterate start
            if len(restricted_verts) > 0:
                print("X", i)
                sorted_idx = restricted_verts.T[1].argsort()
                y_verts = restricted_verts[sorted_idx]
                
                y_start_idx = 0
                for eh, y_edge in enumerate(y_edges): # I NEED TO BE ABLE TO SKIP EDGES WHEN THEY ARE SHIT
                    # How about I increment to the nearest one that it isn't over?
                    # and then deal with the final edge case separately by saying it either has to be over or not the last edge
                    # Or I include the final one and just ignore things that are shit
                    for j, y_vert in enumerate(y_verts[y_start_idx:]):
                        if y_vert[1] > y_edge:
                            restricted_verts = y_verts[y_start_idx:y_start_idx + j]
                            y_start_idx += j

                            # Z iterate start
                            if len(restricted_verts) > 0:
                                print("YY", j)
                                sorted_idx = restricted_verts.T[2].argsort()
                                z_verts = restricted_verts[sorted_idx]

                                z_start_idx = 0
                                for z_edge in z_edges:
                                    for k, z_vert in enumerate(z_verts[z_start_idx:]):
                                        if z_vert[2] > z_edge:
                                            restricted_verts = z_verts[z_start_idx:z_start_idx + k]
                                            z_start_idx += k

                                            if len(restricted_verts) > 0:
                                                print("ZZZ", k)
                                                bboxes.append(Mesh.get_bbox(restricted_verts))
                                                break
                            break
            break
print(time.time() - start)

20 15 7
X 2342
YY 43
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
YY 135
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
X 4966
YY 743
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
YY 1584
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
X 5382
YY 19
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
YY 2874
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
X 6402
YY 3324
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
YY 3
ZZZ 1
ZZZ 1
X 6290
YY 2603
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
YY 592
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
YY 2490
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1929
YY 14
ZZZ 1
ZZZ 1
ZZZ 1
X 23619
YY 4135
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 3320
YY 281
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 181
YY 101
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
YY 2664
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 43
YY 3418
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 105
YY 2263
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 738
YY 3364
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1089
YY 3875
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 3581
YY 2651
ZZZ 1
ZZZ 1
ZZZ 1
X 24616
YY 839
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1
YY 8691
ZZZ 1
ZZZ 1
ZZZ 1
ZZZ 1176
YY 357

In [31]:
bboxes

[array([[222925.5625  , 222925.5625  ],
        [196491.046875, 196491.046875],
        [ 84391.109375,  84391.109375]]),
 array([[222797.53125 , 222797.53125 ],
        [196505.921875, 196505.921875],
        [ 84455.4375  ,  84455.4375  ]]),
 array([[222906.84375 , 222906.84375 ],
        [196391.796875, 196391.796875],
        [ 84475.84375 ,  84475.84375 ]]),
 array([[223002.8125   , 223002.8125   ],
        [196463.703125 , 196463.703125 ],
        [ 84539.5859375,  84539.5859375]]),
 array([[222834.4375  , 222834.4375  ],
        [196393.703125, 196393.703125],
        [ 84566.546875,  84566.546875]]),
 array([[222714.1875, 222714.1875],
        [196453.3125, 196453.3125],
        [ 84588.0625,  84588.0625]]),
 array([[222962.96875  , 222962.96875  ],
        [196432.765625 , 196432.765625 ],
        [ 84637.8046875,  84637.8046875]]),
 array([[222970.765625, 222970.765625],
        [196792.      , 196792.      ],
        [ 84350.859375,  84350.859375]]),
 array([[222869.890625, 

# Old attempts below (with varying degrees of success)
# - The one just below is very fast but misses portions of the mesh, while the one even below that one is entirely correct but is much, much slower.

In [347]:
# Something is off with how I'm choosing the grid split...

class Mesh:
    def __init__(self, vertices, triangles, ignore_isolated_vertices=True):
        self._original_vertices = vertices
        self._original_triangles = triangles
        
        self._vertices = vertices
        self._triangles = triangles
        self._ignore_isolated = ignore_isolated_vertices
        
        self._voxels = list()
    
    class Voxel: # Override some operators to allow direct manipulation to the bbox inside.
        """
        If I really want Voxels that are cubes, I can push the boundaries of the Mesh bounding box to fit what I need (to allow the cubes to fit edge to edge).
        """
        def __init__(self, bbox):
            self._bbox = bbox
            
        @property
        def bbox(self):
            return self._bbox
        
        @bbox.setter
        def bbox(self, bbox):
            if not isinstance(bbox, np.ndarray):
                bbox = np.array(bbox)
            
            if bbox.shape == (3, 2):
                self._bbox = bbox
            else:
                raise ValueError("Bounding box is not in required form which is: array-like with shape (3, 2).")
            
        @property
        def centroid(self):
            return self.bbox.T.mean(axis=0)
        
        def __str__(self):
            return str(self.bbox)
    
    @property
    def voxels(self):
        return self._voxels
    
    @voxels.setter
    def voxels(self, voxels):
        self._voxels = voxels
    
    @property
    def vertices(self): # Could "potentially" have the ignore_isolated_vertices_check in here... but it would mess up indices, so put in
        # another property entirely.
        return self._vertices
    
    @property
    def triangles(self):
        return self._triangles
    
    @property
    def _nonisolated_vertices(self):
        unique_vertex_idx = np.unique(self.triangles)
        return self.vertices[unique_vertex_idx]
    
    @property
    def bbox(self):
        return np.array([(np.min(axis), np.max(axis)) for axis in self.vertices.T])
    
    @staticmethod
    def get_bbox(vertices):
        return np.array([(np.min(axis), np.max(axis)) for axis in vertices.T])
    
        
    def plot_mesh(self, width=1024, height=1024, **kwargs):
        p3.figure(width=width, height=height)
        p3.plot_trisurf(*self.vertices.T/1000, self.triangles, **kwargs)
        p3.squarelim()
        p3.show()
    
    def generate_voxel_lines(self):
        bboxes = np.array([voxel.bbox for voxel in self.voxels])
        return np.array([[[xs[0], ys[0], zs[0]],
                         [xs[0], ys[0], zs[1]],
                         [xs[0], ys[1], zs[1]],
                         [xs[0], ys[1], zs[0]],
                         [xs[0], ys[0], zs[0]],

                         [xs[1], ys[0], zs[0]],
                         [xs[1], ys[0], zs[1]],
                         [xs[1], ys[1], zs[1]],
                         [xs[1], ys[1], zs[0]],
                         [xs[1], ys[0], zs[0]],

                         [xs[0], ys[0], zs[0]],
                         [xs[0], ys[0], zs[1]],
                         [xs[1], ys[0], zs[1]],
                         [xs[1], ys[0], zs[0]],
                         [xs[0], ys[0], zs[0]],

                         [xs[0], ys[1], zs[0]],
                         [xs[0], ys[1], zs[1]],# Something is off with how I'm choosing the grid split...

class Mesh:
    def __init__(self, vertices, triangles, ignore_isolated_vertices=True):
        self._original_vertices = vertices
        self._original_triangles = triangles
        
        self._vertices = vertices
        self._triangles = triangles
        self._ignore_isolated = ignore_isolated_vertices
        
        self._voxels = list()
    
    class Voxel: # Override some operators to allow direct manipulation to the bbox inside.
        """
        If I really want Voxels that are cubes, I can push the boundaries of the Mesh bounding box to fit what I need (to allow the cubes to fit edge to edge).
        """
        def __init__(self, bbox):
            self._bbox = bbox
            
        @property
        def bbox(self):
            return self._bbox
        
        @bbox.setter
        def bbox(self, bbox):
            if not isinstance(bbox, np.ndarray):
                bbox = np.array(bbox)
            
            if bbox.shape == (3, 2):
                self._bbox = bbox
            else:
                raise ValueError("Bounding box is not in required form which is: array-like with shape (3, 2).")
            
        @property
        def centroid(self):
            return self.bbox.T.mean(axis=0)
        
        def __str__(self):
            return str(self.bbox)
    
    @property
    def voxels(self):
        return self._voxels
    
    @voxels.setter
    def voxels(self, voxels):
        self._voxels = voxels
    
    @property
    def vertices(self): # Could "potentially" have the ignore_isolated_vertices_check in here... but it would mess up indices, so put in
        # another property entirely.
        return self._vertices
    
    @property
    def triangles(self):
        return self._triangles
    
    @property
    def _nonisolated_vertices(self):
        unique_vertex_idx = np.unique(self.triangles)
        return self.vertices[unique_vertex_idx]
    
    @property
    def bbox(self):
        return np.array([(np.min(axis), np.max(axis)) for axis in self.vertices.T])
    
    @staticmethod
    def get_bbox(vertices):
        return np.array([(np.min(axis), np.max(axis)) for axis in vertices.T])
    
        
    def plot_mesh(self, width=1024, height=1024, **kwargs):
        p3.figure(width=width, height=height)
        p3.plot_trisurf(*self.vertices.T/1000, self.triangles, **kwargs)
        p3.squarelim()
        p3.show()
    
    def generate_voxel_lines(self):
        bboxes = np.array([voxel.bbox for voxel in self.voxels])
        return np.array([[[xs[0], ys[0], zs[0]],
                         [xs[0], ys[0], zs[1]],
                         [xs[0], ys[1], zs[1]],
                         [xs[0], ys[1], zs[0]],
                         [xs[0], ys[0], zs[0]],

                         [xs[1], ys[0], zs[0]],
                         [xs[1], ys[0], zs[1]],
                         [xs[1], ys[1], zs[1]],
                         [xs[1], ys[1], zs[0]],
                         [xs[1], ys[0], zs[0]],

                         [xs[0], ys[0], zs[0]],
                         [xs[0], ys[0], zs[1]],
                         [xs[1], ys[0], zs[1]],
                         [xs[1], ys[0], zs[0]],
                         [xs[0], ys[0], zs[0]],

                         [xs[0], ys[1], zs[0]],
                         [xs[0], ys[1], zs[1]],
                         [xs[1], ys[1], zs[1]],
                         [xs[1], ys[1], zs[0]],
                         [xs[0], ys[1], zs[0]]] for xs, ys, zs in bboxes])
    
    def plot_voxels(self, width=1024, height=1024):
        fig = p3.figure(width=width, heigh=height)
        [p3.plot(*lines.T/1000, color='blue') for lines in self.generate_voxel_lines()]
#         p3.scatter(*np.array([voxel.centroid for voxel in self.voxels]).T/1000, marker='sphere')
        p3.squarelim()
        p3.show()
        
#         return fig
    
    def plot_mesh_and_voxels(self, width=1024, height=1024, **kwargs):
        p3.figure(width=width, height=height)
        p3.plot_trisurf(*self.vertices.T/1000, self.triangles, **kwargs)
        [p3.plot(*lines.T/1000, color='blue') for lines in self.generate_voxel_lines()]
#         p3.scatter(*np.array([voxel.centroid for voxel in self.voxels]).T/1000, marker='sphere', size=0.5, color='blue')
        p3.squarelim()
        p3.show()    
    
    def merge_touching_voxels(self):
        raise NotImplementedError
        
    def debug(self, voxel_side_length=1000):
        n_splits = np.array([((axis[1] - axis[0]) / voxel_side_length) + 1 for axis in self.bbox], np.uint32)

        x_split, y_split, z_split = [np.linspace(minimum, maximum, num=axis_splits)
                                     for (minimum, maximum), axis_splits in zip(self.bbox, n_splits)]
        
        bboxes = list()
        x_pairs = np.array([(x_split[i], x_split[i+1]) for i in range(len(x_split) - 1)])
        y_pairs = np.array([(y_split[i], y_split[i+1]) for i in range(len(y_split) - 1)])
        z_pairs = np.array([(z_split[i], z_split[i+1]) for i in range(len(z_split) - 1)])
        for xs in x_pairs:
            for ys in y_pairs:
                for zs in z_pairs:
                    bboxes.append((xs, ys, zs))
        bboxes = np.array(bboxes)
        
        bbox_lines = np.array([[[xs[0], ys[0], zs[0]],
                         [xs[0], ys[0], zs[1]],
                         [xs[0], ys[1], zs[1]],
                         [xs[0], ys[1], zs[0]],
                         [xs[0], ys[0], zs[0]],

                         [xs[1], ys[0], zs[0]],
                         [xs[1], ys[0], zs[1]],
                         [xs[1], ys[1], zs[1]],
                         [xs[1], ys[1], zs[0]],
                         [xs[1], ys[0], zs[0]],

                         [xs[0], ys[0], zs[0]],
                         [xs[0], ys[0], zs[1]],
                         [xs[1], ys[0], zs[1]],
                         [xs[1], ys[0], zs[0]],
                         [xs[0], ys[0], zs[0]],

                         [xs[0], ys[1], zs[0]],
                         [xs[0], ys[1], zs[1]],
                         [xs[1], ys[1], zs[1]],
                         [xs[1], ys[1], zs[0]],
                         [xs[0], ys[1], zs[0]]] for xs, ys, zs in bboxes])
        
        p3.figure(width=1024, height=1024)
        p3.plot_trisurf(*self.vertices.T/1000, self.triangles)
        [p3.plot(*lines.T/1000, color='blue') for lines in bbox_lines]
        p3.squarelim()
        p3.show()
        
        return bboxes
        
    def restrict_bboxes(self, voxel_side_length=1000):
        nonisolated_vertices = self._nonisolated_vertices

        n_splits = np.array([((axis[1] - axis[0]) / voxel_side_length) + 1 for axis in self.bbox], np.uint32)
#         n_splits = [11] * 3
    
        x_split, y_split, z_split = [np.linspace(minimum, maximum, num=axis_splits)
                                     for (minimum, maximum), axis_splits in zip(self.bbox, n_splits)]
        
        bboxes = list()
        
        # X iterate start
        sorted_idx = nonisolated_vertices.T[0].argsort()
        x_verts = nonisolated_vertices[sorted_idx]

        x_edge_idx = 1 # See if this has anything to do with the voxel problem
        x_start_idx = 0
        for i, x_vert in tqdm(enumerate(x_verts)):
            if x_vert[0] > x_split[x_edge_idx]:
                x_edge_idx += 1
                restricted_verts = x_verts[x_start_idx:i]
                x_start_idx = i
                                
                # Y iterate start
                if len(restricted_verts) > 0:
                    sorted_idx = restricted_verts.T[1].argsort()
                    y_verts = restricted_verts[sorted_idx]

                    y_edge_idx = 1
                    y_start_idx = 0
                    for j, y_vert in enumerate(y_verts):
                        if y_vert[1] > y_split[y_edge_idx]:
                            y_edge_idx += 1
                            restricted_verts = y_verts[y_start_idx:j]
                            y_start_idx = j
                            
                            # Z iterate start
                            if len(restricted_verts) > 0:
                                sorted_idx = restricted_verts.T[2].argsort()
                                z_verts = restricted_verts[sorted_idx]

                                z_edge_idx = 1
                                z_start_idx = 0
                                for k, z_vert in enumerate(z_verts):
                                    if z_vert[2] > z_split[z_edge_idx]:
                                        z_edge_idx += 1
                                        restricted_verts = z_verts[z_start_idx:k]
                                        z_start_idx = k
                                        
                                        if len(restricted_verts) > 0:
                                            print(i, j, k) # SOME OF THESE BOUNDING BOXES ARE COLLAPSED, EVEN ON EVERY SINGLE AXIS.
                                            print(self.get_bbox(restricted_verts))
                                            bboxes.append(self.get_bbox(restricted_verts))
        return bboxes
    
    def initialize_voxels(self, voxel_side_length=1000):
        bboxes = self.restrict_bboxes(voxel_side_length=voxel_side_length)
        self._voxels = [self.Voxel(bbox) for bbox in bboxes]
        return self.voxels
                         [xs[1], ys[1], zs[1]],
                         [xs[1], ys[1], zs[0]],
                         [xs[0], ys[1], zs[0]]] for xs, ys, zs in bboxes])
    
    def plot_voxels(self, width=1024, height=1024):
        fig = p3.figure(width=width, heigh=height)
        [p3.plot(*lines.T/1000, color='blue') for lines in self.generate_voxel_lines()]
#         p3.scatter(*np.array([voxel.centroid for voxel in self.voxels]).T/1000, marker='sphere')
        p3.squarelim()
        p3.show()
        
#         return fig
    
    def plot_mesh_and_voxels(self, width=1024, height=1024, **kwargs):
        p3.figure(width=width, height=height)
        p3.plot_trisurf(*self.vertices.T/1000, self.triangles, **kwargs)
        [p3.plot(*lines.T/1000, color='blue') for lines in self.generate_voxel_lines()]
#         p3.scatter(*np.array([voxel.centroid for voxel in self.voxels]).T/1000, marker='sphere', size=0.5, color='blue')
        p3.squarelim()
        p3.show()    
    
    def merge_touching_voxels(self):
        raise NotImplementedError
        
    def debug(self, voxel_side_length=1000):
        n_splits = np.array([((axis[1] - axis[0]) / voxel_side_length) + 1 for axis in self.bbox], np.uint32)

        x_split, y_split, z_split = [np.linspace(minimum, maximum, num=axis_splits)
                                     for (minimum, maximum), axis_splits in zip(self.bbox, n_splits)]
        
        bboxes = list()
        x_pairs = np.array([(x_split[i], x_split[i+1]) for i in range(len(x_split) - 1)])
        y_pairs = np.array([(y_split[i], y_split[i+1]) for i in range(len(y_split) - 1)])
        z_pairs = np.array([(z_split[i], z_split[i+1]) for i in range(len(z_split) - 1)])
        for xs in x_pairs:
            for ys in y_pairs:
                for zs in z_pairs:
                    bboxes.append((xs, ys, zs))
        bboxes = np.array(bboxes)
        
        bbox_lines = np.array([[[xs[0], ys[0], zs[0]],
                         [xs[0], ys[0], zs[1]],
                         [xs[0], ys[1], zs[1]],
                         [xs[0], ys[1], zs[0]],
                         [xs[0], ys[0], zs[0]],

                         [xs[1], ys[0], zs[0]],
                         [xs[1], ys[0], zs[1]],
                         [xs[1], ys[1], zs[1]],
                         [xs[1], ys[1], zs[0]],
                         [xs[1], ys[0], zs[0]],

                         [xs[0], ys[0], zs[0]],
                         [xs[0], ys[0], zs[1]],
                         [xs[1], ys[0], zs[1]],
                         [xs[1], ys[0], zs[0]],
                         [xs[0], ys[0], zs[0]],

                         [xs[0], ys[1], zs[0]],
                         [xs[0], ys[1], zs[1]],
                         [xs[1], ys[1], zs[1]],
                         [xs[1], ys[1], zs[0]],
                         [xs[0], ys[1], zs[0]]] for xs, ys, zs in bboxes])
        
        p3.figure(width=1024, height=1024)
        p3.plot_trisurf(*self.vertices.T/1000, self.triangles)
        [p3.plot(*lines.T/1000, color='blue') for lines in bbox_lines]
        p3.squarelim()
        p3.show()
        
        return bboxes
        
    def restrict_bboxes(self, voxel_side_length=1000):
        nonisolated_vertices = self._nonisolated_vertices

        n_splits = np.array([((axis[1] - axis[0]) / voxel_side_length) + 1 for axis in self.bbox], np.uint32)
#         n_splits = [11] * 3
    
        x_split, y_split, z_split = [np.linspace(minimum, maximum, num=axis_splits)
                                     for (minimum, maximum), axis_splits in zip(self.bbox, n_splits)]
        
        bboxes = list()
        
        # X iterate start
        sorted_idx = nonisolated_vertices.T[0].argsort()
        x_verts = nonisolated_vertices[sorted_idx]

        x_edge_idx = 1 # See if this has anything to do with the voxel problem
        x_start_idx = 0
        for i, x_vert in tqdm(enumerate(x_verts)):
            if x_vert[0] > x_split[x_edge_idx]:
                x_edge_idx += 1
                restricted_verts = x_verts[x_start_idx:i]
                x_start_idx = i
                                
                # Y iterate start
                if len(restricted_verts) > 0:
                    sorted_idx = restricted_verts.T[1].argsort()
                    y_verts = restricted_verts[sorted_idx]

                    y_edge_idx = 1
                    y_start_idx = 0
                    for j, y_vert in enumerate(y_verts):
                        if y_vert[1] > y_split[y_edge_idx]:
                            y_edge_idx += 1
                            restricted_verts = y_verts[y_start_idx:j]
                            y_start_idx = j
                            
                            # Z iterate start
                            if len(restricted_verts) > 0:
                                sorted_idx = restricted_verts.T[2].argsort()
                                z_verts = restricted_verts[sorted_idx]

                                z_edge_idx = 1
                                z_start_idx = 0
                                for k, z_vert in enumerate(z_verts):
                                    if z_vert[2] > z_split[z_edge_idx]:
                                        z_edge_idx += 1
                                        restricted_verts = z_verts[z_start_idx:k]
                                        z_start_idx = k
                                        
                                        if len(restricted_verts) > 0:
                                            print(i, j, k) # SOME OF THESE BOUNDING BOXES ARE COLLAPSED, EVEN ON EVERY SINGLE AXIS.
                                            print(self.get_bbox(restricted_verts))
                                            bboxes.append(self.get_bbox(restricted_verts))
        return bboxes
    
    def initialize_voxels(self, voxel_side_length=1000):
        bboxes = self.restrict_bboxes(voxel_side_length=voxel_side_length)
        self._voxels = [self.Voxel(bbox) for bbox in bboxes]
        return self.voxels

SyntaxError: invalid syntax (<ipython-input-347-86e5acdcfaca>, line 102)

In [348]:
mesh = Mesh(fetched_mesh['vertices'], fetched_mesh['triangles'])

In [349]:
# len(mesh.debug(50000))

In [ ]:
start = time.time()
mesh.initialize_voxels(10000)  # SOME OF THESE BOUNDING BOXES ARE COLLAPSED, ON EVERY SINGLE AXIS FOR MANY OF THEM TOO.
# That doesn't fully explain why so many areas are missing bboxes entirely though.
time.time() - start, len(mesh.voxels)

In [ ]:
# mesh.plot_voxels()
mesh.plot_mesh_and_voxels()

In [ ]:
bbox = mesh.bbox

In [ ]:
xs, ys, zs = bbox

In [ ]:
lines = np.array([
    [xs[0], ys[0], zs[0]],
    [xs[0], ys[0], zs[1]],
    [xs[0], ys[1], zs[1]],
    [xs[0], ys[1], zs[0]],
    [xs[0], ys[0], zs[0]],
    
    [xs[1], ys[0], zs[0]],
    [xs[1], ys[0], zs[1]],
    [xs[1], ys[1], zs[1]],
    [xs[1], ys[1], zs[0]],
    [xs[1], ys[0], zs[0]],
    
    [xs[0], ys[0], zs[0]],
    [xs[0], ys[0], zs[1]],
    [xs[1], ys[0], zs[1]],
    [xs[1], ys[0], zs[0]],
    [xs[0], ys[0], zs[0]],
    
    [xs[0], ys[1], zs[0]],
    [xs[0], ys[1], zs[1]],
    [xs[1], ys[1], zs[1]],
    [xs[1], ys[1], zs[0]],
    [xs[0], ys[1], zs[0]]
])

In [ ]:
for i, x in enumerate(xs):
    for j, y in enumerate(ys):
        for k, z in enumerate(zs):
            print(f'[xs[{i}], ys[{j}], zs[{k}]')

In [ ]:
p3.figure(width=1024, height=1024)
p3.plot(*lines.T/1000)
p3.squarelim()
p3.show()

In [350]:
# I can do the full bbox for each plane, then do it across the other planes to reduce the size of the voxels and see what I get.
# I can also do it across multiple restrictions pretty damn quickly with the method I created to partition it out.
# Yeah this will be a very fast and efficient method without worrying about hyperparameterizing the clustering methods.

# Create a 3D grid and scan through each block essentially. Can do it based on planes first, and then further break down the bboxes
# generated by those planes.

# Yeah I can literally just get the bboxes for several rectangular structures findable by the grid partitioning.

# I can turn this into a skeleton by merging touching voxels, or by simple connecting the centroids of voxels by some rule (basically make a tree structure
# starting from one centroid).

# I can merge voxels by checking to see how much their adjacent faces are overlapping (and I should also ensure they don't get too big, by keeping a threshold
# either on the volume/size it can be, or by restricting it by how much empty space I'd be adding in).

class Mesh:
    def __init__(self, vertices, triangles, ignore_isolated_vertices=True):
        self._original_vertices = vertices
        self._original_triangles = triangles
        
        self._vertices = vertices
        self._triangles = triangles
        self._ignore_isolated = ignore_isolated_vertices
        
        self._voxels = list()
    
    class Voxel: # Override some operators to allow direct manipulation to the bbox inside.
        """
        If I really want Voxels that are cubes, I can push the boundaries of the Mesh bounding box to fit what I need (to allow the cubes to fit edge to edge).
        """
        def __init__(self, bbox):
            self._bbox = bbox
            
        @property
        def bbox(self):
            return self._bbox
        
        @bbox.setter
        def bbox(self, bbox):
            if not isinstance(bbox, np.ndarray):
                bbox = np.array(bbox)
            
            if bbox.shape == (3, 2):
                self._bbox = bbox
            else:
                raise ValueError("Bounding box is not in required form which is: array-like with shape (3, 2).")
            
        @property
        def centroid(self):
            return self.bbox.T.mean(axis=0)
        
        def __str__(self):
            return str(self.bbox)
    
    @property
    def voxels(self):
        return self._voxels
    
    @voxels.setter
    def voxels(self, voxels):
        self._voxels = voxels
    
    @property
    def vertices(self): # Could "potentially" have the ignore_isolated_vertices_check in here... but it would mess up indices, so put in
        # another property entirely.
        return self._vertices
    
    @property
    def triangles(self):
        return self._triangles
    
    @property
    def _nonisolated_vertices(self):
        unique_vertex_idx = np.unique(self.triangles)
        return self.vertices[unique_vertex_idx]
    
    @property
    def bbox(self):
        return np.array([(np.min(axis), np.max(axis)) for axis in self.vertices.T])
    
    @staticmethod
    def get_bbox(vertices):
        return np.array([(np.min(axis), np.max(axis)) for axis in vertices.T])
    
        
    def plot_mesh(self, width=1024, height=1024, **kwargs):
        p3.figure(width=width, height=height)
        p3.plot_trisurf(*self.vertices.T/1000, self.triangles, **kwargs)
        p3.squarelim()
        p3.show()
    
    def generate_voxel_lines(self):
        bboxes = np.array([voxel.bbox for voxel in self.voxels])
        return np.array([[[xs[0], ys[0], zs[0]],
                         [xs[0], ys[0], zs[1]],
                         [xs[0], ys[1], zs[1]],
                         [xs[0], ys[1], zs[0]],
                         [xs[0], ys[0], zs[0]],

                         [xs[1], ys[0], zs[0]],
                         [xs[1], ys[0], zs[1]],
                         [xs[1], ys[1], zs[1]],
                         [xs[1], ys[1], zs[0]],
                         [xs[1], ys[0], zs[0]],

                         [xs[0], ys[0], zs[0]],
                         [xs[0], ys[0], zs[1]],
                         [xs[1], ys[0], zs[1]],
                         [xs[1], ys[0], zs[0]],
                         [xs[0], ys[0], zs[0]],

                         [xs[0], ys[1], zs[0]],
                         [xs[0], ys[1], zs[1]],
                         [xs[1], ys[1], zs[1]],
                         [xs[1], ys[1], zs[0]],
                         [xs[0], ys[1], zs[0]]] for xs, ys, zs in bboxes])
    
    def plot_voxels(self, width=1024, height=1024):
        fig = p3.figure(width=width, heigh=height)
        [p3.plot(*lines.T/1000, color='blue') for lines in self.generate_voxel_lines()]
#         p3.scatter(*np.array([voxel.centroid for voxel in self.voxels]).T/1000, marker='sphere')
        p3.squarelim()
        p3.show()
        
#         return fig
    
    def plot_mesh_and_voxels(self, width=1024, height=1024, **kwargs):
        p3.figure(width=width, height=height)
        p3.plot_trisurf(*self.vertices.T/1000, self.triangles, **kwargs)
        [p3.plot(*lines.T/1000, color='blue') for lines in self.generate_voxel_lines()]
#         p3.scatter(*np.array([voxel.centroid for voxel in self.voxels]).T/1000, marker='sphere', size=0.5, color='blue')
        p3.squarelim()
        p3.show()   
    
    def merge_touching_voxels(self):
        raise NotImplementedError
            
        # Someone said that looping over numpy arrays should be avoided... can this be done more efficiently in some way?
#         corner_coords = list()
#         for x in x_split:
#             for y in y_split:
#                 for z in z_split:
#                     corner_coords.append((x, y, z))
#         corner_coords = np.array(corner_coords)
    
    # Turn these splits into bboxes so I can turn them into Voxels
    # I need pairwise bboxes for all of these
    # I should hmm. I was going to say to compute the centroids for these "rectangles" and then add the radius, but they're rectangles.
    # First thing I need is all of the points at which will be the corners for the rectangles.
    # Or I can potentially look at the walls formed and check from there.
    def grid_split(self, num_splits_each_axis=10):
        """
        :param num_splits_each_axis: 'automated' just means it'll be computed based on the volume of the mesh bbox.
        For now it just defaults to 10 per axis (so 1000 initial bboxes).
        """
        x_split, y_split, z_split = [np.linspace(minimum, maximum, num=num_splits_each_axis) for minimum, maximum in self.bbox]
        
        # Make the ratio such that I get more cubic voxels.        
        bboxes = list()
        x_pairs = np.array([(x_split[i], x_split[i+1]) for i in range(len(x_split) - 1)])
        y_pairs = np.array([(y_split[i], y_split[i+1]) for i in range(len(y_split) - 1)])
        z_pairs = np.array([(z_split[i], z_split[i+1]) for i in range(len(z_split) - 1)])
        for xs in x_pairs:
            for ys in y_pairs:
                for zs in z_pairs:
                    bboxes.append((xs, ys, zs))
        bboxes = np.array(bboxes)
        
        return bboxes
    
    def OLD_initialize_voxels(self, num_splits_each_axis=10):
        bboxes = self.grid_split(num_splits_each_axis=num_splits_each_axis)
        self._voxels = [self.Voxel(bbox) for bbox in bboxes]
        return self.voxels
    
    def OLDER_restrict_bboxes(self, num_splits_each_axis=10):
        voxels = self.OLD_initialize_voxels(num_splits_each_axis=num_splits_each_axis)
        nonisolated_vertices = self._nonisolated_vertices
        t_nonisolated_vertices = nonisolated_vertices.T
#         x_sorted_idx = t_nonisolated_vertices[0].argsort()
#         y_sorted_idx = t_nonisolated_vertices[1].argsort()
#         z_sorted_idx = t_nonisolated_vertices[2].argsort()
        
#         x_verts = nonisolated_vertices[x_sorted_idx]
#         y_verts = nonisolated_vertices[y_sorted_idx]
#         z_verts = nonisolated_vertices[z_sorted_idx]
        
        x_axis, y_axis, z_axis = t_nonisolated_vertices
        
        # I can make this even faster still! Which is extremely necessary as the time complexity of the number of Voxels to make increases.
        # Less voxels might still be what I want though.
        
        masks = list()
        not_empty_voxels = list()
        for i, voxel in tqdm(enumerate(voxels)):
            # Now I need to use each voxel to retrieve the vertices within its bbox, then reduce the voxel's bbox to the bbox of the vertices present within it.
            # First retrive the vertices:
            (x_min, x_max), (y_min, y_max), (z_min, z_max) = voxel.bbox
            x_mask = np.all(((x_axis >= x_min), (x_axis <= x_max)), axis=0)
            y_mask = np.all(((y_axis >= y_min), (y_axis <= y_max)), axis=0)
            z_mask = np.all(((z_axis >= z_min), (z_axis <= z_max)), axis=0)
            mask = np.all((x_mask, y_mask, z_mask), axis=0)
            if mask.sum() > 0:
                # Next get the bbox of the vertices.
                relevant_verts = nonisolated_vertices[mask]
                new_bbox = self.get_bbox(relevant_verts)
                # Now insert the new bbox into the voxel object.
                voxel.bbox = new_bbox
                not_empty_voxels.append(i)
                
        self.voxels = np.array(voxels)[not_empty_voxels]
        
    """
    def thick_plane(self, num_planes=50):
        z_space = np.linspace(*self.bbox[2], num=num_planes)

        starting_idx = 0
        partition_edge_idx = list([starting_idx])
        verts = self._sorted_vertices
        for j, z_edge in enumerate(z_space[1:-1]):
            for i, vert in enumerate(verts.T[2][starting_idx:]):
                if vert > z_edge:
                    starting_idx += i
                    partition_edge_idx.append(starting_idx)
                    break
        partition_edge_idx.append(-1) #(len(verts) - 1)
        
        return np.array(partition_edge_idx)
    """
    def OLD_restrict_bboxes(self, voxel_side_length=1000):
    #         voxels = self.initialize_voxels(num_splits_each_axis=num_splits_each_axis)
        nonisolated_vertices = self._nonisolated_vertices
        t_nonisolated_vertices = nonisolated_vertices.T

        n_splits = np.array([((axis[1] - axis[0]) / voxel_side_length) + 1 for axis in self.bbox], np.uint32)

        x_split, y_split, z_split = [np.linspace(minimum, maximum, num=axis_splits)
                                     for (minimum, maximum), axis_splits in zip(self.bbox, n_splits)]

    #         bboxes = list()
    #         x_pairs = np.array([(x_split[i], x_split[i+1]) for i in range(len(x_split) - 1)])
    #         y_pairs = np.array([(y_split[i], y_split[i+1]) for i in range(len(y_split) - 1)])
    #         z_pairs = np.array([(z_split[i], z_split[i+1]) for i in range(len(z_split) - 1)])
    #         for xs in x_pairs:
    #             for ys in y_pairs:
    #                 for zs in z_pairs:
    #                     bboxes.append((xs, ys, zs))
    #         bboxes = np.array(bboxes)

        # iterate through x_verts
        x_sorted_idx = t_nonisolated_vertices[0].argsort()
        x_verts = nonisolated_vertices[x_sorted_idx]
        # Can probably iterate through x_verts until I find the next x_edge in x_split. And just cycle through x_split.

        x_edge = x_split[1]
        voxel_box = list()
        count = 0
        
        print(x_split, y_split, z_split)
        
        x_starting_idx = 0
        for x_edge in tqdm(x_split[1:-1]):
            current_verts = list()
            for i, x_vert in enumerate(x_verts[x_starting_idx:]):
                if x_vert[0] <= x_edge:
                    current_verts.append(x_vert)
                else:
                    x_starting_idx += i
                    
                    print("X", len(current_verts))
                    
                    if len(current_verts) > 0:
                        current_verts = np.array(current_verts)
                        y_sorted_idx = current_verts.T[1].argsort()
                        y_verts = current_verts[y_sorted_idx]
                                                
                        y_starting_idx = 0
                        for y_edge in y_split[1:-1]: # y_edge might not be what I want?
                            current_verts = list()
                            for j, y_vert in enumerate(y_verts[y_starting_idx:]):
                                if y_vert[1] <= y_edge:
#                                     print(y_vert)
                                    current_verts.append(y_vert)
                                else:
                                    y_starting_idx += j
                
                                    print("Y", len(current_verts))
                                    
                                    if len(current_verts) > 0:
                                        current_verts = np.array(current_verts)
                                        z_sorted_idx = current_verts.T[2].argsort()
                                        z_verts = current_verts[z_sorted_idx]

                                        z_starting_idx = 0
                                        for z_edge in z_split[1:-1]: # BE CAREFUL WITH THAT -1 INDEX THERE
                                            current_verts = list()
                                            for k, z_vert in enumerate(z_verts[y_starting_idx:]):
                                                if z_vert[2] <= z_edge:
                                                    print(z_vert)
                                                    current_verts.append(z_vert)
                                                else:
                                                    z_starting_idx += k
                                                    
                                                    print("Z", len(current_verts))
                                                    
#                                                     print(current_verts)
                                                    print()
                                                    voxel_box.append(self.get_bbox(np.array(current_verts)))
        return voxel_box
        
#         for x_vert in x_verts: # Go through these sorted vertices, then once you have one split block for x's, sort by y.
                
#             if x_vert[0] <= x_edge:
#                 count += 1
#             else:
#                 print(count, x_edge)
#                 break

    #         for x_edge in x_split:


        # then iterate through sorted y_verts from there


        # then iterate through z_verts


        # Make sure I get cubic voxels more or less.
        
    def debug(self, voxel_side_length=1000):
        n_splits = np.array([((axis[1] - axis[0]) / voxel_side_length) + 1 for axis in self.bbox], np.uint32)

        x_split, y_split, z_split = [np.linspace(minimum, maximum, num=axis_splits)
                                     for (minimum, maximum), axis_splits in zip(self.bbox, n_splits)]
        
        bboxes = list()
        x_pairs = np.array([(x_split[i], x_split[i+1]) for i in range(len(x_split) - 1)])
        y_pairs = np.array([(y_split[i], y_split[i+1]) for i in range(len(y_split) - 1)])
        z_pairs = np.array([(z_split[i], z_split[i+1]) for i in range(len(z_split) - 1)])
        for xs in x_pairs:
            for ys in y_pairs:
                for zs in z_pairs:
                    bboxes.append((xs, ys, zs))
        bboxes = np.array(bboxes)
        
        return bboxes
        
    def restrict_bboxes(self, voxel_side_length=1000):
        nonisolated_vertices = self._nonisolated_vertices

#         n_splits = np.array([((axis[1] - axis[0]) / voxel_side_length) + 1 for axis in self.bbox], np.uint32)
        n_splits = [50] * 3
    
        x_split, y_split, z_split = [np.linspace(minimum, maximum, num=axis_splits)
                                     for (minimum, maximum), axis_splits in zip(self.bbox, n_splits)]
        
        eughhhh = list()
        
        # X iterate start
        sorted_idx = nonisolated_vertices.T[0].argsort()
        x_verts = nonisolated_vertices[sorted_idx]

        x_edge_idx = 1
        x_start_idx = 0
        for i, x_vert in tqdm(enumerate(x_verts)):
            if x_vert[0] > x_split[x_edge_idx]:
                x_edge_idx += 1
                restricted_verts = x_verts[x_start_idx:i]
                x_start_idx = i
                                
                # Y iterate start
                if len(restricted_verts) > 0:
                    sorted_idx = restricted_verts.T[1].argsort()
                    y_verts = restricted_verts[sorted_idx]

                    y_edge_idx = 1
                    y_start_idx = 0
                    for j, y_vert in enumerate(y_verts):
                        if y_vert[1] > y_split[y_edge_idx]:
                            y_edge_idx += 1
                            restricted_verts = y_verts[y_start_idx:j]
                            y_start_idx = j
                            
                            # Z iterate start
                            if len(restricted_verts) > 0:
                                sorted_idx = restricted_verts.T[2].argsort()
                                z_verts = restricted_verts[sorted_idx]

                                z_edge_idx = 1
                                z_start_idx = 0
                                for k, z_vert in enumerate(z_verts):
                                    if z_vert[2] > z_split[z_edge_idx]:
                                        z_edge_idx += 1
                                        restricted_verts = z_verts[z_start_idx:k]
                                        z_start_idx = k
                                                                                
                                        if len(restricted_verts) > 0:
                                            eughhhh.append(self.get_bbox(restricted_verts))
        return eughhhh
    
    def initialize_voxels(self, voxel_side_length=1000):
        bboxes = self.restrict_bboxes(voxel_side_length=voxel_side_length)
        self._voxels = [self.Voxel(bbox) for bbox in bboxes]
        return self.voxels

In [351]:
mesh = Mesh(fetched_mesh['vertices'], fetched_mesh['triangles'])

In [352]:
# len(mesh.debug(50000))

In [353]:
start = time.time()
mesh.OLDER_restrict_bboxes(20)  # SOME OF THESE BOUNDING BOXES ARE COLLAPSED, ON EVERY SINGLE AXIS FOR MANY OF THEM TOO.
# That doesn't fully explain why so many areas are missing bboxes entirely though.
time.time() - start, len(mesh.voxels)



















0it [00:00, ?it/s]

















28it [00:00, 271.85it/s]

















55it [00:00, 268.87it/s]

















86it [00:00, 278.44it/s]

















119it [00:00, 291.00it/s]

















152it [00:00, 299.87it/s]

















185it [00:00, 306.27it/s]

















218it [00:00, 312.99it/s]

















252it [00:00, 318.42it/s]

















285it [00:00, 318.90it/s]

















316it [00:01, 314.30it/s]

















347it [00:01, 308.68it/s]

















378it [00:01, 306.86it/s]

















411it [00:01, 312.32it/s]

















443it [00:01, 310.32it/s]

















474it [00:01, 308.56it/s]

















505it [00:01, 303.45it/s]

















540it [00:01, 313.92it/s]

















572it [00:01, 309.16it/s]

















603it [00:01, 308.79it/s]

















636it [00:02, 313.08it/s]

















669it [00:02, 316.90it/s]

















701it [00:02, 314.91it/s]

















733

2591it [00:08, 255.40it/s]

















2617it [00:08, 249.42it/s]

















2643it [00:09, 245.67it/s]

















2668it [00:09, 244.56it/s]

















2693it [00:09, 238.79it/s]

















2717it [00:09, 226.90it/s]

















2743it [00:09, 234.77it/s]

















2771it [00:09, 244.77it/s]

















2801it [00:09, 257.81it/s]

















2833it [00:09, 272.58it/s]

















2866it [00:09, 286.24it/s]

















2896it [00:09, 280.75it/s]

















2925it [00:10, 277.95it/s]

















2954it [00:10, 277.30it/s]

















2982it [00:10, 269.15it/s]

















3010it [00:10, 265.10it/s]

















3038it [00:10, 268.56it/s]

















3065it [00:10, 265.75it/s]

















3094it [00:10, 272.22it/s]

















3122it [00:10, 273.67it/s]

















3151it [00:10, 277.01it/s]

















3185it [00:11, 291.71it/s]

















3219it [00:11, 304.20it/s]







(23.53341841697693, 410)

In [354]:
# mesh.plot_voxels()
mesh.plot_mesh_and_voxels()

In [ ]:
bbox = mesh.bbox

In [ ]:
print(np.ceil(bbox[0][0]/bbox[0][1]))

bbox[0][0], bbox[0][1]

In [ ]:
voxel_side_length = 1000

In [ ]:
ratio_x = bbox[0][0] / bbox[0][1]
ratio_y = bbox[1][0] / bbox[1][1]
ratio_z = bbox[2][0] / bbox[2][1]

In [ ]:
dif_x = bbox[0][1] - bbox[0][0]
dif_y = bbox[1][1] - bbox[1][0]
dif_z = bbox[2][1] - bbox[2][0]

In [ ]:
voxel_side_length = 1000
n_splits = np.zeros(3, np.int32)
for i, axis in enumerate(bbox):
    n_splits[i] = ((axis[1] - axis[0]) / voxel_side_length) + 1

In [ ]:
n_splits

In [ ]:
n_splits

In [ ]:
dif_x, dif_y, dif_z

In [ ]:
x, y, z = 4, 5, 3

In [ ]:
int((dif_x / 1000) + 1), int((dif_y / 1000) + 1), int((dif_z / 1000) + 1)

In [ ]:
x, y, z

In [ ]:
np.linspace(bbox[0][0], bbox[0][1])

In [ ]:
mesh.plot_mesh_and_voxels()

In [ ]:
mesh.plot_voxels()

In [ ]:
mesh.plot_mesh()

In [ ]:
# What I really want to be able to do, is to shrink the bbox of the Mesh down to fit the (nonisolated) vertices.
# From there it should split the resulting complex shape into cubes..

# So to start, I need to get bboxes on a chosen plane at each layer (like the whole bbox).
# So do that several times for, say, the xy plane at each z block.
# Then I can start looking at each "slice" and get inner bounding boxes.

# Due to the odd shape of some of these neurons and their parts, I might really just want to use rectangular shapes that can be rotated...
# but at that point wouldn't it make more sense to create skeletons? If I figure out how to do that it would make my life (and getting
# accurate minimum distance measures) much easier. Though it would be difficult to extract the info needed for those skeletons.

# I can do the full bbox for each plane, then do it across the other planes to reduce the size of the voxels and see what I get.
# I can also do it across multiple restrictions pretty damn quickly with the method I created to partition it out.
# Yeah this will be a very fast and efficient method without worrying about hyperparameterizing the clustering methods.

# Create a 3D grid and scan through each block essentially. Can do it based on planes first, and then further break down the bboxes
# generated by those planes.

# Yeah I can literally just get the bboxes for several rectangular structures findable by the grid partitioning.

class Mesh:
    def __init__(self, vertices, triangles):
        self._original_vertices = vertices
        self._original_triangles = triangles
        
        self._vertices = vertices
        self._triangles = triangles
    
    @staticmethod
    def get_bbox(vertices):
        np.array([(np.min(axis), np.max(axis)) for axis in vertices.T])
    
    def grid_split(self, num_splits_each_axis=50):
        splits = [np.linspace(minimum, maximum, num=num_splits_each_axis) for minimum, maximum in self.bbox]
        return splits        
        
    def merge_small_voxels(self):
        pass
    
    class Voxel:
        def __init__(self, centroid, radius):
            self._x, self._y, self._z = centroid
            self._radius = radius
            
        @property
        def centroid(self):
            return np.array((self._x, self._y, self._z))
        
        @property
        def radius(self):
            return self._radius
        
        @radius.setter
        def radius(self, radius):
            self._radius = radius
            
        @property
        def bbox(self):
            # Just add and subtract the radius from the centroid vector
            return np.vstack((self.centroid - self.radius, self.centroid + self.radius)).T
    
    @property
    def vertices(self):
        return self._vertices
    
    @property
    def triangles(self):
        return self._triangles
    
    @property
    def _nonisolated_vertices(self):
        unique_vertex_idx = np.unique(self.triangles)
        return self.vertices[unique_vertex_idx]
    
    @property
    def _sorted_vertices(self):
        """
        Returns sorted nonisolated vertices. This is best used when only dealing with vertices and not triangles too.
        """
        raise NotImplementedError
        
        verts = self._nonisolated_vertices
        sorted_idx = verts.T[2].argsort()
        return verts[sorted_idx]
    
    @property
    def bbox(self):
        return np.array([(np.min(axis), np.max(axis)) for axis in self.vertices.T]) # Should I use nonisolated vertices though?
    
    def plot_mesh(self, width=1024, height=1024, **kwargs):
        p3.figure(width=width, height=height)
        p3.plot_trisurf(*self.vertices.T/1000, self.triangles, **kwargs)
        p3.squarelim()
        p3.show()
    
    def plot_verts(self, width=1024, height=1024, targeted_verts=None, **kwargs):
        if targeted_verts is None:
            verts = self.vertices
        else:
            verts = targeted_verts
        
        p3.figure(width=width, height=height)
        p3.scatter(*verts.T/1000, **kwargs)
        p3.squarelim()
        p3.show()
        
    def get_adjacent_voxels(self):
        
        # I just need to look at how the Voxel bboxes touch other Voxel bboxes
        
        raise NotImplementedError
    
    # Should I make this a lazy generator?
    def partition(self, axis, bbox_view=None, num_partitions=50):
        """
        Splits up the vertices along an axis.
        :param axis: The axis along which to partition the vertices.
        :param view: BBox view which will be used to look at a specific portion of the mesh. Leave as None to partition the entire mesh.
        :param num_partitions: Number of partitions/slices to be returned.
        """
        
        if bbox_view is None:
            axis_space = np.linspace(self.bbox[axis], num=num_planes)
            starting_idx = 0
            partition_edge_idx = list([starting_idx])
        else:
            raise NotImplementedError("bbox_view hasn't been implemented yet, bitch")
            axis_space = np.linspace(bbox_view[axis], num=num_planes)
            starting_idx = 0 # Would have to figure out if I want to keep indices based on the total _sorted_vertices or...
            partition_edge_idx = list([starting_idx])
            
        nonisolated_verts = self._nonisolated_vertices
        sorted_idx = nonisolated_verts.T[axis].argsort()
        axis_verts = nonisolated_verts[sorted_idx].T[axis]
        
        
        
    def thick_plane(self, num_planes=50):
        z_space = np.linspace(*self.bbox[2], num=num_planes)

        starting_idx = 0
        partition_edge_idx = list([starting_idx])
        verts = self._sorted_vertices
        for j, z_edge in enumerate(z_space[1:-1]):
            for i, vert in enumerate(verts.T[2][starting_idx:]):
                if vert > z_edge:
                    starting_idx += i
                    partition_edge_idx.append(starting_idx)
                    break
        partition_edge_idx.append(-1) #(len(verts) - 1)
        
        return np.array(partition_edge_idx)
    
    # Should still go through with this approach regardless.
    # Might either want to look into density or hierarchical clustering methods. But try to do a simple (note: efficient) version.
    # Could be accomplished by choosing a voxel center, then expanding outwards until there is a density drought. Then choose another voxel
    # center nearby.
    # Also the center of these Voxel bboxes should be between the 2 edges being looked at.
    def get_bbox_of_plane(self, num_planes=50):
        part_idx = self.thick_plane(num_planes)
        verts = self._sorted_vertices
        
        for i in range(len(part_idx) - 1):
            start = time.time()
            lower_idx, upper_idx = part_idx[i], part_idx[i+1]
            targets = verts[lower_idx:upper_idx]
            
            plt.scatter(*targets.mean(axis=0)[0:2])
            
#             find_n_clusters = PCA()
#             find_n_clusters.fit(targets)
#             return find_n_clusters.components_
        
#             clusterer = KMeans(n_clusters=4)
#             clusterer.fit(targets)
            
#             print(time.time() - start)
#             plt.scatter(*targets.T[0:2], c=clusterer.labels_, cmap="Set1")
#             plt.hist(verts[lower_idx:upper_idx].T[0], bins=50, color=(i/50, i/100, i/50, 0.9))
#             plt.show()
#             print(time.time() - start)
            
#         self.plot_verts(targeted_verts=verts, size=0.1, marker='sphere')
        
#         plt.hist(verts.T[0])
#         plt.show()
#         plt.hist(verts.T[1])
#         plt.show()
#         plt.hist(verts.T[2])
#         plt.show()

In [ ]:
mesh = Mesh(fetched_mesh['vertices'], fetched_mesh['triangles'])

In [ ]:
mesh.get_bbox_of_plane()

In [ ]:
mesh.get_bbox_of_plane()

In [ ]:
mesh.plot_ipv()

In [ ]:
part_idx = mesh.thick_plane()

In [ ]:
part_idx

In [ ]:
mesh._sorted_vertices[part_idx[-1]], mesh.bbox[-1][-1]

In [ ]:
mesh._sorted_vertices[-1]

In [ ]:
voxel = mesh.Voxel((1, 3, 5), 10)

In [ ]:
print(voxel.centroid)
print(voxel.radius)
print(voxel.bbox)

In [ ]:
(ta3p100.AllenSomaClass() & 'cell_class="glia"') - ta3p100.SegmentExclude

In [ ]:
ta3p100.AllenSomaClass()